In [3]:
import cv2
import numpy as np
import os
from itertools import chain
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import load_model

In [4]:
class Custom_generator(Sequence):
    def __init__(self, src_path, input_shape, class_num, target, batch_size, is_train=False):
        folder_list = os.listdir(src_path)
        self.src_path = src_path
        self.is_train = is_train
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.x, self.y = [], []

        # with open("./datasets/wnids.txt", "r") as f:
        #     cls_list = f.readlines()
        # cls_list = [cls_object.replace("\n", "") for cls_object in cls_list]

        # for _, folder in enumerate(folder_list):
        #     # imgs = glob.glob(src_path+folder+"/*.JPEG")
        #     imgs = os.listdir(src_path)
        #     # cls = cls_list.index(folder)
        #     self.x += imgs
        #     self.y += list(np.full(((len(imgs)), class_num), fill_value=target))
        imgs = os.listdir(src_path)
        self.x += imgs
        self.y += list(np.full(((len(imgs)), class_num), fill_value=target))

        self.on_epoch_end()

    def on_epoch_end(self):
        self.index = np.arange(len(self.x))
        if self.is_train:
            np.random.shuffle(self.index)

    def __len__(self):
        return round(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x, batch_y = [], []
        batch_index = self.index[idx*self.batch_size:(idx+1)*self.batch_size]
        for i in batch_index:
            batch_x.append(self.x[i])
            batch_y.append(self.y[i])
        out_x, out_y = self.__data_gen(batch_x, batch_y)
        return out_x, out_y

    def __data_gen(self, x, y):
        input_x = np.zeros((self.batch_size, self.input_shape[0], self.input_shape[1], self.input_shape[2]), dtype=np.float32)
        # copy_input_x = np.array([])
        input_y = []
        for idx in range(len(x)):
            img = cv2.imread(f'{self.src_path}/{x[idx]}')
            input_x[idx] = cv2.resize(img, (self.input_shape[0], self.input_shape[1]))
            input_x[idx] = cv2.cvtColor(input_x[idx], cv2.COLOR_BGR2HSV)
            input_x[idx] = np.asarray(input_x[idx], np.float32)
            # copy_input_x[idx] = input_x[idx].copy()
            # input_x[idx][:,:,0] = copy_input_x[:,:,0] / 180.
            # input_x[idx][:,:,1] = copy_input_x[:,:,1] / 255.
            # input_x[idx][:,:,2] = copy_input_x[:,:,2] / 255.

            input_x[idx][:,:,0] = input_x[idx][:,:,0] / 180.
            input_x[idx][:,:,1] = input_x[idx][:,:,1] / 255.
            input_x[idx][:,:,2] = input_x[idx][:,:,2] / 255.

        input_y = np.asarray(y, dtype=np.float32)
        return input_x, input_y

In [5]:
custom_gen_normal = Custom_generator(src_path='./model_images/DA4649_rotate/train/1_normal',
                                     input_shape=(400, 400, 3), target = np.array([0, 0, 0, 0, 0, 1], dtype=np.float32), class_num = 6, batch_size=32, is_train=True)
custom_gen_center = Custom_generator('./model_images/DA4649_rotate/train/2_center_rotate_per30',
                                     (400, 400, 3), target = np.array([0, 0, 0, 0, 1, 0], dtype=np.float32), class_num = 6, batch_size=32, is_train=True)
custom_gen_colorpoor = Custom_generator('./model_images/DA4649_rotate/train/3_colorpoor_rotate_per5',
                                        (400, 400, 3), target = np.array([0, 0, 0, 1, 0, 0], dtype=np.float32), class_num = 6, batch_size=32, is_train=True)
custom_gen_dotmissing = Custom_generator('./model_images/DA4649_rotate/train/4_dotmissing_rotate_per5',
                                         (400, 400, 3), target = np.array([0, 0, 1, 0, 0, 0], dtype=np.float32), class_num = 6, batch_size=32, is_train=True)
custom_gen_inkcut = Custom_generator('./model_images/DA4649_rotate/train/5_inkcut_rotate_per15',
                                     (400, 400, 3), target = np.array([0, 1, 0, 0, 0, 0], dtype=np.float32), class_num = 6, batch_size=32, is_train=True)
custom_gen_line = Custom_generator('./model_images/DA4649_rotate/train/6_line_rotate_per60',
                                   (400, 400, 3), target = np.array([1, 0, 0, 0, 0, 0], dtype=np.float32), class_num = 6, batch_size=32, is_train=True)

train_gen_x = chain(custom_gen_normal, custom_gen_center, custom_gen_colorpoor, custom_gen_dotmissing, custom_gen_inkcut, custom_gen_line)

normal_y = np.full((len(os.listdir('./model_images/DA4649_rotate/train/1_normal')), 6), fill_value= np.array([0, 0, 0, 0, 0, 1]))
center_y = np.full((len(os.listdir('./model_images/DA4649_rotate/train/2_center_rotate_per30')), 6), fill_value= np.array([0, 0, 0, 0, 1, 0]))
colorpoor_y = np.full((len(os.listdir('./model_images/DA4649_rotate/train/3_colorpoor_rotate_per5')), 6), fill_value= np.array([0, 0, 0, 1, 0, 0]))
dotmissing_y = np.full((len(os.listdir('./model_images/DA4649_rotate/train/4_dotmissing_rotate_per5')), 6), fill_value= np.array([0, 0, 1, 0, 0, 0]))
inkcut_y = np.full((len(os.listdir('./model_images/DA4649_rotate/train/5_inkcut_rotate_per15')), 6), fill_value= np.array([0, 1, 0, 0, 0, 0]))
line_y = np.full((len(os.listdir('./model_images/DA4649_rotate/train/6_line_rotate_per60')), 6), fill_value= np.array([1, 0, 0, 0, 0, 0]))

In [6]:
# model = load_model('../predict_models/DA4649_defect.h5')

In [7]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten
from tensorflow.keras.models import Model

def create_poweradeNet(shape, classes):
    input_ = Input(shape=shape)

    before_layer = Conv2D(filters=64, kernel_size=(3, 3), strides=1, activation='relu')(input_)
    before_layer = Conv2D(filters=64, kernel_size=(5, 5), strides=1, activation='relu')(before_layer)
    before_layer = MaxPooling2D(pool_size=2)(before_layer)
    before_layer = BatchNormalization()(before_layer)

    flatten_layer = Flatten()(before_layer)
    # Fully Connection
    fc1 = Dense(units=1, activation='relu')(flatten_layer)
    fc2 = Dense(units=1, activation='relu')(fc1)
    output = Dense(units=classes, activation='softmax')(fc2)
    model = Model(inputs=[input_], outputs=[output])

    return model

In [10]:
model = create_poweradeNet(shape=(400, 400, 3), classes=6)
model.compile(loss='categorical_crossentropy')
model.fit(train_gen_x, batch_size=32, epochs=1)

    367/Unknown - 142s 365ms/step - loss: 1.4879

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,6] labels_size=[27,6]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at \AppData\Local\Temp\ipykernel_10812\301034582.py:3) ]] [Op:__inference_train_function_1420]

Function call stack:
train_function
